In [85]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML
from sklearn.metrics import mean_absolute_error

df_user_item_rating_train = pd.read_csv(".\\ml-100k_u1\\u1.base",delimiter='\t',names=['user_id','item_id','rating','timestamp'])
df_item_rating_train = df_user_item_rating_train.groupby('item_id',as_index=False)['rating'].mean()

df_user_item_rating_test = pd.read_csv(".\\ml-100k_u1\\u1.test",delimiter='\t',names=['user_id','item_id','rating','timestamp'])
df_item_rating_test = df_user_item_rating_test.groupby('item_id',as_index=False)['rating'].mean()

df_items = pd.read_csv(".\\ml-100k_u1\\u.item",delimiter='|',names=['item_id','movie title','release date','video release date','IMDb URL'
          ,'unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western'])



merged_item_info_rating_train = pd.merge(df_item_rating_train,df_items)
merged_item_info_rating_test = pd.merge(df_item_rating_test,df_items)

df_train_x_train = merged_item_info_rating_train[['unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']]

df_train_y_train = merged_item_info_rating_train[['item_id','rating']].set_index(['item_id'])

df_train_x_test = merged_item_info_rating_test[['unknown','Action','Adventure','Animation','Childrens','Comedy',
          'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
        'Thriller','War','Western']]

df_train_y_test = merged_item_info_rating_test[['item_id','rating']].set_index(['item_id'])


model = LinearRegression()
model.fit(df_train_x_train, df_train_y_train)
model.score(df_train_x_train, df_train_y_train)

y_pred = model.predict(df_train_x_test)

df_train_y_test['prediction'] = y_pred

MAE = mean_absolute_error(df_train_y_test['rating'], df_train_y_test['prediction'])
display(MAE)



0.70268042237201733

In [132]:
sorted_df_train_y_test = df_train_y_test.sort_values(by = 'prediction',ascending=False)



# df_train_y_test.index
lst = {}
lst_test_actual = {}
predicted_recommand = {}
random_recommand = {} 

user_precision_predicted = []
user_recall_predicted = []
user_precision_random = []
user_recall_random = []

user_data_frame = []


for user in df_user_item_rating_test['user_id'].unique():
    lst[user] = df_user_item_rating_train.loc[df_user_item_rating_train['user_id'] == user]['item_id']
    predicted_recommand[user] = sorted_df_train_y_test.loc[~sorted_df_train_y_test.index.isin(lst[user])].head(20).index
    random_recommand[user] = sorted_df_train_y_test.sample(20).index
    lst_test_actual[user] = df_user_item_rating_test.loc[df_user_item_rating_test['user_id'] == user]['item_id']
    user_precision_predicted.append(len(set(lst_test_actual[user]).intersection(predicted_recommand[user])) / 20.0)
    user_recall_predicted.append(len(set(lst_test_actual[user]).intersection(predicted_recommand[user])) / (len(lst_test_actual[user]) * 1.0))
    user_precision_random.append(len(set(lst_test_actual[user]).intersection(random_recommand[user])) / 20.0)
    user_recall_random.append(len(set(lst_test_actual[user]).intersection(random_recommand[user])) / (len(lst_test_actual[user]) * 1.0))
    user_data_frame.append(user)

raw_data = {}
raw_data['user_id'] = user_data_frame
raw_data['precision_predicted'] = user_precision_predicted
raw_data['recall_predicted'] = user_recall_predicted
raw_data['precision_random'] = user_precision_random
raw_data['recall_random'] = user_recall_random

df_analitics = pd.DataFrame(data = raw_data,index = [user_data_frame],columns = ['user_id','precision_predicted','recall_predicted','precision_random','recall_random'])
df_analitics = df_analitics.set_index(['user_id'])
display(df_analitics)



,precision_predicted,recall_predicted,precision_random,recall_random
user_id,,,,
1,0.20,0.029197,0.05,0.007299
2,0.10,0.090909,0.00,0.000000
3,0.05,0.038462,0.10,0.076923
4,0.05,0.100000,0.00,0.000000
5,0.05,0.011905,0.05,0.011905
6,0.20,0.039604,0.00,0.000000
7,0.35,0.036842,0.10,0.010526
8,0.10,0.068966,0.00,0.000000
9,0.05,0.100000,0.00,0.000000
